In [1]:
library(GEOquery)
library(limma)
library(AnnotationDbi)
library(stringr)

Caricamento del pacchetto richiesto: Biobase

Caricamento del pacchetto richiesto: BiocGenerics


Caricamento pacchetto: ‘BiocGenerics’


I seguenti oggetti sono mascherati da ‘package:stats’:

    IQR, mad, sd, var, xtabs


I seguenti oggetti sono mascherati da ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Setting options('download.file.method.GEOquery'='auto')

Setting options('GEOquery.inmemory.gpl'=FALSE)


Caricamento pacchett

In [2]:
dataPath = "./Data/"

In [3]:
# Checklist:
#  - Colonne senza nome
#  - Colonne con valori nulli
#  - Colonne con doppio gene (tipicamente separate da ///)
#  - Colonne in minuscolo da portare in maiuscolo
#  - Traduzione nomi colonne

# A1A

In [74]:
csv_name = 'A1A_deficiency_GSE109516_bal.csv'

In [75]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [76]:
head(df)

,FAM138F,OR4F5,OR4F3,LOC729737,DDX11L1,WASH7P,LOC100132062,LOC100133331,OR4F3.1,LOC100133331.1,⋯,DUX4L2.3,DUX2.14,DUX4.10,DUX4L,DUX4.11,DUX4L5.2,DUX2.15,DUX4.12,DUX4L2.4,DUX2.16
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X02S015.02B9055_006,0,0,0,1.202460,0.0000000,14.33000,0.000341173,0.926014,0,7.667140,⋯,0,0,0,0,0,0,0,0,0,0
X02S028.02B9271_001,0,0,0,0.587269,0.0587284,4.23908,0.000121072,0.369643,0,0.648837,⋯,0,0,0,0,0,0,0,0,0,0
X03S009.03B9062_012,0,0,0,0.336415,0.0000000,4.10042,0.000159783,0.199500,0,0.217815,⋯,0,0,0,0,0,0,0,0,0,0
X03S039.03B9206_009,0,0,0,0.761621,0.4863730,2.87126,0.000103373,0.227027,0,0.400471,⋯,0,0,0,0,0,0,0,0,0,0
X06S017.06B9129_013,0,0,0,1.319410,0.0891071,4.70020,0.000111671,0.228535,0,0.400137,⋯,0,0,0,0,0,0,0,0,0,0
X06S039.06B9229_001,0,0,0,0.989486,0.0000000,6.13913,0.000133185,0.564246,0,1.083000,⋯,0,0,0,0,0,0,0,0,0,0


In [77]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [78]:
head(df)

,FAM138F,OR4F5,OR4F3,LOC729737,DDX11L1,WASH7P,LOC100132062,LOC100133331,OR4F3.1,LOC100133331.1,⋯,TTTY3B,DAZ4.3,DAZ3.2,DAZ2.1,CDY1B.1,CDY1,SPRY3.1,IL9R.1,VAMP7.1,UTY
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X02S015.02B9055_006,0,0,0,1.202460,0.0000000,14.33000,0.000341173,0.926014,0,7.667140,⋯,0,0,0,0,0,0,0.2088980,0.123211,0.0343559,0.194620
X02S028.02B9271_001,0,0,0,0.587269,0.0587284,4.23908,0.000121072,0.369643,0,0.648837,⋯,0,0,0,0,0,0,0.0964487,0.116302,1.8701700,0.163447
X03S009.03B9062_012,0,0,0,0.336415,0.0000000,4.10042,0.000159783,0.199500,0,0.217815,⋯,0,0,0,0,0,0,0.0418214,0.189153,1.8015300,0.172999
X03S039.03B9206_009,0,0,0,0.761621,0.4863730,2.87126,0.000103373,0.227027,0,0.400471,⋯,0,0,0,0,0,0,0.0000000,0.000000,1.5274400,6.207220
X06S017.06B9129_013,0,0,0,1.319410,0.0891071,4.70020,0.000111671,0.228535,0,0.400137,⋯,0,0,0,0,0,0,0.0351214,0.211891,1.7779800,0.149813
X06S039.06B9229_001,0,0,0,0.989486,0.0000000,6.13913,0.000133185,0.564246,0,1.083000,⋯,0,0,0,0,0,0,0.0527176,0.158957,0.9176300,1.572170


In [79]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

In [80]:
csv_name = 'A1A_deficiency_GSE109516_pbmc.csv'

In [81]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [82]:
head(df)

,FAM138F,OR4F5,OR4F3,LOC729737,DDX11L1,WASH7P,LOC100132062,LOC100133331,OR4F3.1,LOC100133331.1,⋯,DUX4L2.3,DUX2.14,DUX4.10,DUX4L,DUX4.11,DUX4L5.2,DUX2.15,DUX4.12,DUX4L2.4,DUX2.16
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X02S015.02S7041_013,0,0,0,5.13813,0.908668,7.33859,0.000512406,1.00044e+00,0,1.44256,⋯,0,0,0,0,0,0,0,0,0,0
X02S028.02S7283_003,0,0,0,5.08120,2.572830,11.11950,0.000176911,1.79337e+00,0,1.24944,⋯,0,0,0,0,0,0,0,0,0,0
X03S009.03S7065_011,0,0,0,3.04018,2.247790,7.13176,1.199710000,4.51872e-05,0,1.18256,⋯,0,0,0,0,0,0,0,0,0,0
X06S017.06S7136_006,0,0,0,2.20487,3.338000,7.00880,0.000144586,8.53105e-01,0,1.16502,⋯,0,0,0,0,0,0,0,0,0,0
X06S039.06S7324_011,0,0,0,8.03142,1.421980,8.12425,1.915370000,2.33545e-05,0,2.11526,⋯,0,0,0,0,0,0,0,0,0,0
X04S047.04S7473_012RUN248,0,0,0,4.92161,1.297020,6.28220,1.988100000,3.89329e-04,0,1.42607,⋯,0,0,0,0,0,0,0,0,0,0


In [83]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [84]:
head(df)

,FAM138F,OR4F5,OR4F3,LOC729737,DDX11L1,WASH7P,LOC100132062,LOC100133331,OR4F3.1,LOC100133331.1,⋯,CSPG4P1Y.1,DAZ4.3,DAZ3.2,DAZ2.1,CDY1B.1,CDY1,SPRY3.1,IL9R.1,VAMP7.1,UTY
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X02S015.02S7041_013,0,0,0,5.13813,0.908668,7.33859,0.000512406,1.00044e+00,0,1.44256,⋯,0,0,0,0,0,0,0.1085300,0.245382,1.99787,0.120459
X02S028.02S7283_003,0,0,0,5.08120,2.572830,11.11950,0.000176911,1.79337e+00,0,1.24944,⋯,0,0,0,0,0,0,0.0510752,0.167969,2.27362,0.223543
X03S009.03S7065_011,0,0,0,3.04018,2.247790,7.13176,1.199710000,4.51872e-05,0,1.18256,⋯,0,0,0,0,0,0,0.0597672,0.111284,2.12879,0.248822
X06S017.06S7136_006,0,0,0,2.20487,3.338000,7.00880,0.000144586,8.53105e-01,0,1.16502,⋯,0,0,0,0,0,0,0.0529292,0.136767,1.64396,0.185010
X06S039.06S7324_011,0,0,0,8.03142,1.421980,8.12425,1.915370000,2.33545e-05,0,2.11526,⋯,0,0,0,0,0,0,0.0406139,0.183653,1.80640,5.414140
X04S047.04S7473_012RUN248,0,0,0,4.92161,1.297020,6.28220,1.988100000,3.89329e-04,0,1.42607,⋯,0,0,0,0,0,0,0.0407886,0.102468,2.41275,9.628940


In [85]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# MCM

In [103]:
csv_name = 'MCM_GSE149607.csv'

In [104]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [105]:
head(df)

,NR_046018,NR_024368,NR_029406,XR_132471,NR_047526,NM_152486,NM_198317,NM_005101,NM_198576,NR_038869,⋯,XM_003403688.1,NR_049896.2,XR_133519.1,XR_132462.2,NM_012149,NR_024086,XR_112193,XM_001719777,XR_133541,NM_203348
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM4505813,8.426488,10.62813,6.012424,4.399510,6.294647,5.329514,5.893486,6.627709,5.476104,5.010456,⋯,4.829462,4.008998,7.144405,4.161195,5.772545,3.434427,2.853097,4.039324,4.736561,5.138100
GSM4505814,8.994904,10.73934,6.323561,4.002165,6.346021,5.162432,5.258358,6.525893,5.103988,4.864387,⋯,4.872092,3.228349,6.719474,4.174266,5.723040,3.480727,3.023857,3.695276,5.053663,5.087892
GSM4505815,9.470577,11.32028,6.266544,3.916870,5.699336,5.684052,5.583466,7.286742,5.555661,5.377296,⋯,4.958282,3.734034,7.155735,3.832421,6.147892,3.232951,2.950671,4.222785,5.302822,5.630389
GSM4505816,9.005315,10.68989,6.029893,4.055520,6.114030,5.361383,5.428657,6.356181,5.056486,4.871640,⋯,4.758407,3.199611,7.359878,4.234975,5.538055,3.132322,3.330705,4.347722,4.862523,5.046917
GSM4505817,8.914529,10.82360,6.430465,4.013163,6.156909,4.780517,5.606540,6.698649,5.313160,5.458017,⋯,4.719867,3.272944,6.884299,4.240961,5.549666,3.603027,3.253354,3.792074,4.950836,4.845475
GSM4505818,9.102384,11.26057,6.298412,3.986049,5.839989,5.111051,5.737249,6.876548,5.268361,4.971873,⋯,4.773473,3.233161,6.324773,4.482434,5.567406,3.345491,3.567779,3.909546,5.048886,5.417264


In [106]:
library(org.Hs.eg.db)

In [107]:
keys <- keys(org.Hs.eg.db)
ref_gene <- select(org.Hs.eg.db, keys=keys, columns = c("SYMBOL", "REFSEQ"))

'select()' returned 1:many mapping between keys and columns



In [112]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("REFSEQ")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="REFSEQ")

In [113]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$REFSEQ == i, 'SYMBOL'])
}

In [115]:
colnames(df) <- new_col

In [117]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [124]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [125]:
head(df)

,DDX11L1,LINC01000,RPL23AP87,LINC01128,SAMD11,KLHL17,ISG15,AGRN,LINC01342,MIR200B,⋯,MIR4632,MIR4641,MIR4658,MIR4747,MIR4453,MIR4469,MIR4793,DUX5,COPG2IT1,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM4505813,8.426488,10.62813,6.012424,4.399510,6.294647,5.329514,5.893486,6.627709,5.476104,5.010456,⋯,2.346176,2.971787,7.256904,3.009726,4.273818,1.620877,4.332614,3.713831,3.028895,4.116191
GSM4505814,8.994904,10.73934,6.323561,4.002165,6.346021,5.162432,5.258358,6.525893,5.103988,4.864387,⋯,2.353896,3.829023,7.240062,3.241480,4.624471,1.228825,4.828307,3.978035,3.320887,3.944383
GSM4505815,9.470577,11.32028,6.266544,3.916870,5.699336,5.684052,5.583466,7.286742,5.555661,5.377296,⋯,2.695717,2.831540,6.579659,3.148770,5.029887,1.786814,4.949089,3.775315,3.200742,4.229219
GSM4505816,9.005315,10.68989,6.029893,4.055520,6.114030,5.361383,5.428657,6.356181,5.056486,4.871640,⋯,2.104197,2.997496,7.122319,3.614771,4.392521,1.640545,5.086360,3.540823,2.924842,4.536970
GSM4505817,8.914529,10.82360,6.430465,4.013163,6.156909,4.780517,5.606540,6.698649,5.313160,5.458017,⋯,2.388887,3.727300,7.238576,3.275639,4.587617,1.989560,4.799526,3.869301,2.695159,3.760633
GSM4505818,9.102384,11.26057,6.298412,3.986049,5.839989,5.111051,5.737249,6.876548,5.268361,4.971873,⋯,1.758543,3.267482,6.871074,3.263139,5.194161,1.464511,4.441410,3.915283,2.765299,4.325420


In [126]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# Monogenic diabetes

In [127]:
csv_name = 'Monogenic_diabetes_GSE188827.csv'

In [128]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [129]:
head(df)

,MIR6859.1,MIR1302.2HG,MIR1302.2,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,RNU6.1100P,⋯,MT.ND4,MT.TH,MT.TS2,MT.TL2,MT.ND5,MT.ND6,MT.TE,MT.CYB,MT.TT,MT.TP
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
X6A._2D...30582.001.R1...Total.counts,0,5,0,0,1,38,0,0,414,0,⋯,36146,1,27,1,28946,1912,11,10502,2,1
X6D_2D...30582.002.R1...Total.counts,0,42,0,0,0,35,0,0,300,0,⋯,30024,0,11,0,29539,2416,5,12480,2,1
X6F_2D...30582.003.R1...Total.counts,0,13,0,1,12,43,0,0,485,0,⋯,48557,6,17,0,43190,1921,13,22531,1,0


In [139]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [140]:
head(df)

,MIR1302.2HG,FAM138A,OR4F5,AL627309.1,AL627309.5,AL627309.4,AP006222.2,AL732372.1,OR4F29,AC114498.1,⋯,MT.ND4,MT.TH,MT.TS2,MT.TL2,MT.ND5,MT.ND6,MT.TE,MT.CYB,MT.TT,MT.TP
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
X6A._2D...30582.001.R1...Total.counts,5,0,1,38,414,2,10,9,10,1,⋯,36146,1,27,1,28946,1912,11,10502,2,1
X6D_2D...30582.002.R1...Total.counts,42,0,0,35,300,3,10,0,3,0,⋯,30024,0,11,0,29539,2416,5,12480,2,1
X6F_2D...30582.003.R1...Total.counts,13,1,12,43,485,2,7,0,2,0,⋯,48557,6,17,0,43190,1921,13,22531,1,0


In [141]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# TPMT

In [142]:
csv_name = 'TPMT_deficiency_GSE105074_colon.csv'

In [148]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [152]:
head(df)

,EEF1A1,GAPDH,RPS28,IPO13,CDT1,LPP,FLJ36848,HNRNPAB,LOH12CR1,SNIP1,⋯,MIR1323,MIR21,MIR708,MIR1206,MIR326,MIR212,MIR33B,MIR1915,SNORD114.2,NCRNA00173
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM2817541,13.64481,12.40319,10.38449,7.590015,7.912756,11.27456,7.171634,12.04086,6.826576,8.342355,⋯,6.837659,6.812695,6.881034,6.849228,6.739612,6.853754,6.831605,6.904387,6.962242,6.937679
GSM2817542,13.96538,12.25442,10.03633,7.648298,7.622234,11.41729,7.321077,12.13418,6.792153,8.314609,⋯,6.932993,6.978373,6.953917,7.125832,6.981325,6.808266,7.042811,7.126413,7.162367,6.942887
GSM2817543,13.57105,12.32100,10.12464,7.911876,7.419815,11.42628,6.983852,11.94789,6.892973,8.321148,⋯,6.922442,6.917750,6.961512,6.819651,6.746120,6.719426,6.817881,6.678560,7.044067,6.872669
GSM2817544,13.78871,12.07161,10.30719,7.640305,7.696911,11.44371,7.431970,12.23676,6.788516,8.203980,⋯,6.884778,6.814788,6.851034,6.911141,6.849615,6.795676,6.854515,6.849070,7.046170,6.875422
GSM2817545,13.71651,12.19751,10.16028,8.126870,7.127398,11.50464,6.909118,11.66210,6.732931,8.527528,⋯,7.166601,6.987090,6.941556,6.795586,6.998717,6.896027,6.811875,7.317758,6.863090,7.195713
GSM2817546,13.69787,12.24746,10.54064,7.542460,7.726057,11.12604,7.195175,11.97288,6.875122,8.613182,⋯,6.803078,6.895575,7.027187,6.909986,6.763764,6.942405,6.879047,6.910719,7.068710,6.974410


In [159]:
df <- df[1:11,]

In [161]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [162]:
head(df)

,EEF1A1,GAPDH,RPS28,IPO13,CDT1,LPP,FLJ36848,HNRNPAB,LOH12CR1,SNIP1,⋯,MIR1323,MIR21,MIR708,MIR1206,MIR326,MIR212,MIR33B,MIR1915,SNORD114.2,NCRNA00173
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM2817541,13.64481,12.40319,10.38449,7.590015,7.912756,11.27456,7.171634,12.04086,6.826576,8.342355,⋯,6.837659,6.812695,6.881034,6.849228,6.739612,6.853754,6.831605,6.904387,6.962242,6.937679
GSM2817542,13.96538,12.25442,10.03633,7.648298,7.622234,11.41729,7.321077,12.13418,6.792153,8.314609,⋯,6.932993,6.978373,6.953917,7.125832,6.981325,6.808266,7.042811,7.126413,7.162367,6.942887
GSM2817543,13.57105,12.32100,10.12464,7.911876,7.419815,11.42628,6.983852,11.94789,6.892973,8.321148,⋯,6.922442,6.917750,6.961512,6.819651,6.746120,6.719426,6.817881,6.678560,7.044067,6.872669
GSM2817544,13.78871,12.07161,10.30719,7.640305,7.696911,11.44371,7.431970,12.23676,6.788516,8.203980,⋯,6.884778,6.814788,6.851034,6.911141,6.849615,6.795676,6.854515,6.849070,7.046170,6.875422
GSM2817545,13.71651,12.19751,10.16028,8.126870,7.127398,11.50464,6.909118,11.66210,6.732931,8.527528,⋯,7.166601,6.987090,6.941556,6.795586,6.998717,6.896027,6.811875,7.317758,6.863090,7.195713
GSM2817546,13.69787,12.24746,10.54064,7.542460,7.726057,11.12604,7.195175,11.97288,6.875122,8.613182,⋯,6.803078,6.895575,7.027187,6.909986,6.763764,6.942405,6.879047,6.910719,7.068710,6.974410


In [163]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

In [164]:
csv_name = 'TPMT_deficiency_GSE105074_rectum.csv'

In [165]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [166]:
df

,EEF1A1,GAPDH,RPS28,IPO13,CDT1,LPP,FLJ36848,HNRNPAB,LOH12CR1,SNIP1,⋯,MIR1323,MIR21,MIR708,MIR1206,MIR326,MIR212,MIR33B,MIR1915,SNORD114.2,NCRNA00173
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM2817526,13.96538,12.02243,10.30342,7.923415,7.556815,11.54259,7.031143,12.13677,6.762560,8.475246,⋯,6.902072,6.958248,6.858053,6.980707,6.833720,6.785075,6.912212,6.892806,6.954553,7.111827
GSM2817527,13.88178,12.15410,10.03943,7.713827,7.421822,11.07283,7.124354,12.03808,6.785807,8.735668,⋯,6.851239,7.115920,6.996154,6.854219,6.760976,6.934950,6.733048,6.751993,6.829305,6.917215
GSM2817528,13.79709,11.83830,10.45016,7.649561,7.387937,12.09938,6.982091,11.77892,6.870814,8.559494,⋯,6.867476,7.083544,6.912047,6.850638,6.971292,6.680617,6.840213,6.903527,7.049400,7.181285
GSM2817529,13.97765,12.22559,10.53693,7.774665,7.656395,11.54900,7.415311,12.01088,6.765795,8.444221,⋯,6.827497,6.887409,6.979913,6.868392,6.826196,6.805928,7.101132,6.806975,6.986882,7.011594
GSM2817530,13.69787,12.22000,10.56426,7.905209,7.513871,11.48225,7.203571,12.19846,6.873852,8.182619,⋯,6.817047,7.063463,6.807679,6.936486,6.789320,6.854278,6.791942,6.862519,6.862885,6.942666
GSM2817531,13.71680,12.40056,10.60377,7.724124,7.463802,11.85733,7.144013,11.81693,6.907503,8.530834,⋯,6.848729,7.525166,6.850436,6.793189,6.835550,6.955170,6.810018,6.764952,7.005092,7.054844
GSM2817532,13.89530,12.04633,10.33024,7.673532,7.631012,11.61793,7.108502,12.00602,6.848512,8.670686,⋯,6.858568,7.019461,6.906992,6.979434,6.952082,6.765160,6.884458,6.778028,6.925981,6.933213
GSM2817533,13.70983,11.86142,10.41843,7.632611,7.300464,11.50210,7.349908,12.19065,6.893657,8.581532,⋯,6.755555,6.981054,6.862114,6.993875,6.886333,6.674160,6.829705,7.138849,6.944981,7.048155
GSM2817534,13.73369,11.88192,10.05838,7.833626,7.158525,11.64098,7.093148,11.99469,6.897108,8.234841,⋯,6.802975,7.084788,6.944243,6.915160,6.902676,6.757125,6.843316,6.872361,6.987333,6.978397


In [168]:
df <- df[-c(10,14),]

In [169]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [170]:
head(df)

,EEF1A1,GAPDH,RPS28,IPO13,CDT1,LPP,FLJ36848,HNRNPAB,LOH12CR1,SNIP1,⋯,MIR1323,MIR21,MIR708,MIR1206,MIR326,MIR212,MIR33B,MIR1915,SNORD114.2,NCRNA00173
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM2817526,13.96538,12.02243,10.30342,7.923415,7.556815,11.54259,7.031143,12.13677,6.762560,8.475246,⋯,6.902072,6.958248,6.858053,6.980707,6.833720,6.785075,6.912212,6.892806,6.954553,7.111827
GSM2817527,13.88178,12.15410,10.03943,7.713827,7.421822,11.07283,7.124354,12.03808,6.785807,8.735668,⋯,6.851239,7.115920,6.996154,6.854219,6.760976,6.934950,6.733048,6.751993,6.829305,6.917215
GSM2817528,13.79709,11.83830,10.45016,7.649561,7.387937,12.09938,6.982091,11.77892,6.870814,8.559494,⋯,6.867476,7.083544,6.912047,6.850638,6.971292,6.680617,6.840213,6.903527,7.049400,7.181285
GSM2817529,13.97765,12.22559,10.53693,7.774665,7.656395,11.54900,7.415311,12.01088,6.765795,8.444221,⋯,6.827497,6.887409,6.979913,6.868392,6.826196,6.805928,7.101132,6.806975,6.986882,7.011594
GSM2817530,13.69787,12.22000,10.56426,7.905209,7.513871,11.48225,7.203571,12.19846,6.873852,8.182619,⋯,6.817047,7.063463,6.807679,6.936486,6.789320,6.854278,6.791942,6.862519,6.862885,6.942666
GSM2817531,13.71680,12.40056,10.60377,7.724124,7.463802,11.85733,7.144013,11.81693,6.907503,8.530834,⋯,6.848729,7.525166,6.850436,6.793189,6.835550,6.955170,6.810018,6.764952,7.005092,7.054844


In [171]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# SCDO

In [4]:
csv_name = 'SCDO_GSE116930.csv'

In [5]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [8]:
head(df)

,LUCIFERASE,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,⋯,AL022397.1,ZNF48.1,AL022067.1,EXOC3L2.1,RP13.210D15.9,WDFY4.1,RP11.244E17.1,CTB.60B18.23,RP3.454G6.2,RP5.937E21.8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PSM_HES7_KO_1,1379.390,102.8180,3.34310,70.6159,3.54495,11.9621,0.1980390,0.0000000,42.7183,40.4207,⋯,0.347405,8.27968,16.8887,6.73088,0,0,1.386850,0.000000,0.0634917,0.0000000
PSM_HES7_KO_7,2072.330,100.2600,3.85374,73.8619,3.12265,12.1027,0.1147530,0.0000000,45.3055,50.1169,⋯,0.567233,10.29990,10.4985,3.64435,0,0,0.954315,0.316007,0.0000000,0.0627254
PSM_HES7_KO_8,2261.150,95.7013,2.46920,65.5125,3.59161,10.5953,0.2664440,0.0152127,41.5188,49.0845,⋯,0.412645,9.78573,13.7200,5.32349,0,0,1.046150,0.000000,0.0000000,0.0000000
PSM_DLL3_KO_2,355.957,82.6633,2.68993,61.2381,4.52932,13.1934,0.1217250,0.0000000,46.9607,45.0787,⋯,0.922770,8.23142,10.3666,3.88345,0,0,1.212070,0.120381,0.0000000,0.0000000
PSM_DLL3_KO_6,501.639,93.8430,3.60968,63.6134,3.29774,10.9561,0.0874466,0.0000000,47.6370,43.4710,⋯,0.651967,8.86148,13.9170,4.76608,0,0,1.908110,0.000000,0.0000000,0.0512468
PSM_LFNG_KO_2,745.421,89.7061,4.14642,64.9405,4.49170,11.5493,0.2743050,0.0000000,47.0909,44.7843,⋯,0.915244,7.45420,13.9584,4.27964,0,0,1.186880,0.000017,0.0000000,0.0000000


In [9]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [10]:
head(df)

,LUCIFERASE,TSPAN6,TNMD,DPM1,SCYL3,C1ORF112,FGR,CFH,FUCA2,GCLC,⋯,RP11.402P6.15,AC074141.1,AL022397.1,ZNF48.1,AL022067.1,EXOC3L2.1,RP11.244E17.1,CTB.60B18.23,RP3.454G6.2,RP5.937E21.8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PSM_HES7_KO_1,1379.390,102.8180,3.34310,70.6159,3.54495,11.9621,0.1980390,0.0000000,42.7183,40.4207,⋯,0.1163280,0.482393,0.347405,8.27968,16.8887,6.73088,1.386850,0.000000,0.0634917,0.0000000
PSM_HES7_KO_7,2072.330,100.2600,3.85374,73.8619,3.12265,12.1027,0.1147530,0.0000000,45.3055,50.1169,⋯,0.0000000,0.590162,0.567233,10.29990,10.4985,3.64435,0.954315,0.316007,0.0000000,0.0627254
PSM_HES7_KO_8,2261.150,95.7013,2.46920,65.5125,3.59161,10.5953,0.2664440,0.0152127,41.5188,49.0845,⋯,0.0435326,0.607555,0.412645,9.78573,13.7200,5.32349,1.046150,0.000000,0.0000000,0.0000000
PSM_DLL3_KO_2,355.957,82.6633,2.68993,61.2381,4.52932,13.1934,0.1217250,0.0000000,46.9607,45.0787,⋯,0.0000000,0.272975,0.922770,8.23142,10.3666,3.88345,1.212070,0.120381,0.0000000,0.0000000
PSM_DLL3_KO_6,501.639,93.8430,3.60968,63.6134,3.29774,10.9561,0.0874466,0.0000000,47.6370,43.4710,⋯,0.0000000,0.269360,0.651967,8.86148,13.9170,4.76608,1.908110,0.000000,0.0000000,0.0512468
PSM_LFNG_KO_2,745.421,89.7061,4.14642,64.9405,4.49170,11.5493,0.2743050,0.0000000,47.0909,44.7843,⋯,0.0000000,0.783665,0.915244,7.45420,13.9584,4.27964,1.186880,0.000017,0.0000000,0.0000000


In [11]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# PNH 1

In [52]:
csv_name = 'PNH_GSE83808.csv'

In [53]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [56]:
head(df)

,100,10000,10001,10005,10006,10007,10009,10010,100101267,100101467,⋯,9985,9986,9987,9988,9989,9990,9991,9993,9994,9997
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PNH1.CD4Memory,9.09317,53.6941,15.9824,8.41726,32.8238,8.02703,16.1475,34.6507,11.2298,2.58559,⋯,4.89987,10.97420,77.6561,10.60100,15.11080,14.57890,49.7691,31.3169,18.7174,7.24178
PNH1.CD4Naive,6.16933,40.7802,17.3133,5.57398,25.6378,6.36067,16.2447,12.7740,10.3504,3.62134,⋯,4.48913,5.88211,83.2055,13.86300,4.16826,24.16650,33.9694,31.5107,19.5255,2.82557
PNH1.CD8Memory,11.24090,65.5781,18.7741,8.90167,35.5291,10.34080,13.5210,31.6277,12.4404,2.99128,⋯,9.13052,10.93570,71.1315,5.21605,16.46420,9.82878,48.1389,28.1670,16.7845,6.13246
PNH1.CD8Naive,6.15602,48.8477,19.3960,9.26614,31.9750,13.01360,14.6299,18.1740,10.6911,4.52836,⋯,5.05769,9.67363,86.7307,11.00270,6.69589,18.49940,38.6927,29.9686,18.8735,3.36891
PNH2.CD4Memory,11.49760,45.2357,21.8686,7.09714,37.3964,4.48016,17.6756,27.5051,13.7323,1.81018,⋯,11.12550,10.09490,69.2918,13.22850,12.89410,18.63430,55.0993,22.3891,15.4246,4.58083
PNH2.CD4Naive,10.96020,38.0731,22.9002,6.88371,34.9433,4.80480,26.3469,18.9634,14.5415,3.18990,⋯,4.84352,7.00930,94.1287,14.34070,6.12798,27.30520,49.6073,24.6074,19.5232,1.10126


In [55]:
colnames(df) <- sub("X", "", colnames(df))

In [57]:
library(org.Hs.eg.db)

In [58]:
keys <- keys(org.Hs.eg.db)
ref_gene <- select(org.Hs.eg.db, keys=keys, columns = c("SYMBOL", "ENTREZID"))

'select()' returned 1:1 mapping between keys and columns



In [59]:
head(ref_gene)

,ENTREZID,SYMBOL
,<chr>,<chr>
1,1,A1BG
2,2,A2M
3,3,A2MP1
4,9,NAT1
5,10,NAT2
6,11,NATP


In [60]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("ENTREZID")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="ENTREZID")

In [61]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$ENTREZID == i, 'SYMBOL'])
}

In [62]:
colnames(df) <- new_col

In [64]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [66]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [67]:
head(df)

,ADA,AKT3,MED6,ACOT8,ABI1,GNPDA1,ZBTB33,TANK,POM121C,ZSCAN30,⋯,RCE1,HNRNPDL,DMTF1,PPP4R1,SLC12A6,PTBP3,DGCR2,CASP8AP2,SCO2,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PNH1.CD4Memory,9.09317,53.6941,15.9824,8.41726,32.8238,8.02703,16.1475,34.6507,11.2298,2.58559,⋯,17.11950,139.446,11.42530,9.94935,4.89987,10.97420,77.6561,10.60100,15.11080,14.57890
PNH1.CD4Naive,6.16933,40.7802,17.3133,5.57398,25.6378,6.36067,16.2447,12.7740,10.3504,3.62134,⋯,15.43850,109.524,4.12501,12.95290,4.48913,5.88211,83.2055,13.86300,4.16826,24.16650
PNH1.CD8Memory,11.24090,65.5781,18.7741,8.90167,35.5291,10.34080,13.5210,31.6277,12.4404,2.99128,⋯,14.43630,132.421,10.65590,9.25604,9.13052,10.93570,71.1315,5.21605,16.46420,9.82878
PNH1.CD8Naive,6.15602,48.8477,19.3960,9.26614,31.9750,13.01360,14.6299,18.1740,10.6911,4.52836,⋯,13.95110,135.498,5.99272,10.11610,5.05769,9.67363,86.7307,11.00270,6.69589,18.49940
PNH2.CD4Memory,11.49760,45.2357,21.8686,7.09714,37.3964,4.48016,17.6756,27.5051,13.7323,1.81018,⋯,7.53992,142.924,10.96370,9.69707,11.12550,10.09490,69.2918,13.22850,12.89410,18.63430
PNH2.CD4Naive,10.96020,38.0731,22.9002,6.88371,34.9433,4.80480,26.3469,18.9634,14.5415,3.18990,⋯,6.62334,146.930,6.55586,14.31150,4.84352,7.00930,94.1287,14.34070,6.12798,27.30520


In [68]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# PNH 2

In [106]:
csv_name = 'PNH_GSE122352.csv'

In [107]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [108]:
df

,hsa.let.7a.5p,hsa.let.7b.5p,hsa.let.7c.5p,hsa.let.7d.3p,hsa.let.7d.5p,hsa.let.7e.5p,hsa.let.7f.5p,hsa.let.7g.5p,hsa.let.7i.5p,hsa.miR.101.3p,⋯,hsa.miR.92a.3p,hsa.miR.93.5p,hsa.miR.99a.5p,hsa.miR.99b.5p,mmu.miR.378a.3p,UniSp2,UniSp3.IPC,UniSp4,UniSp5,UniSp6
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
GSM3464496,0.633,0.245,0.033,0.334,0.054,0.074,0.235,0.651,0.279,1.292,⋯,7.362,0.959,0.001,0.044,0.096,NA,NA,NA,NA,NA
GSM3464497,0.868,0.177,0.043,0.221,0.091,0.050,0.309,0.549,0.189,0.724,⋯,1.759,0.755,0.012,0.065,0.035,NA,NA,NA,NA,NA
GSM3464498,1.597,0.258,0.060,0.452,0.120,0.000,0.471,0.990,0.340,2.181,⋯,5.560,1.306,0.032,0.009,0.137,NA,NA,NA,NA,NA
GSM3464499,1.899,0.898,0.100,0.609,0.044,0.005,0.298,1.145,1.218,4.908,⋯,22.085,3.127,0.080,0.008,0.613,NA,NA,NA,NA,NA
GSM3464500,1.454,0.514,0.091,0.390,0.069,0.039,0.514,0.877,0.270,2.657,⋯,5.618,1.682,0.035,0.013,0.113,NA,NA,NA,NA,NA
GSM3464501,1.439,0.549,0.053,0.399,0.102,0.010,0.340,1.075,0.648,4.611,⋯,11.511,1.847,0.042,0.050,0.214,NA,NA,NA,NA,NA
GSM3464502,1.558,0.441,0.038,0.536,0.081,0.049,0.321,0.779,0.379,3.506,⋯,8.515,2.479,0.014,0.017,0.130,NA,NA,NA,NA,NA
GSM3464503,1.444,0.463,0.062,0.418,0.086,0.012,0.401,0.859,0.199,1.223,⋯,3.138,1.454,0.023,0.008,0.084,NA,NA,NA,NA,NA
GSM3464504,2.250,0.559,0.112,0.993,0.165,0.037,0.503,1.301,0.953,4.000,⋯,25.457,2.549,0.053,0.013,0.432,NA,NA,NA,NA,NA


In [109]:
library(miRBaseConverter)

In [110]:
res = miRNA_NameToAccession(gsub("\\.", "-", colnames(df)))

In [111]:
colnames(df) <- res$Accession

In [113]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [115]:
head(df)

,MIMAT0000062,MIMAT0000063,MIMAT0000064,MIMAT0004484,MIMAT0000065,MIMAT0000066,MIMAT0000067,MIMAT0000414,MIMAT0000415,MIMAT0000099,⋯,MIMAT0002177,MIMAT0002817,MIMAT0003249,MIMAT0003258,MIMAT0003338,MIMAT0000092,MIMAT0000093,MIMAT0000097,MIMAT0000689,MIMAT0003151
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM3464496,0.633,0.245,0.033,0.334,0.054,0.074,0.235,0.651,0.279,1.292,⋯,3.387,0.030,0.007,0.255,0.316,7.362,0.959,0.001,0.044,0.096
GSM3464497,0.868,0.177,0.043,0.221,0.091,0.050,0.309,0.549,0.189,0.724,⋯,0.862,0.013,0.008,0.204,0.138,1.759,0.755,0.012,0.065,0.035
GSM3464498,1.597,0.258,0.060,0.452,0.120,0.000,0.471,0.990,0.340,2.181,⋯,1.809,0.032,0.014,0.495,0.568,5.560,1.306,0.032,0.009,0.137
GSM3464499,1.899,0.898,0.100,0.609,0.044,0.005,0.298,1.145,1.218,4.908,⋯,9.613,0.002,0.004,1.032,0.584,22.085,3.127,0.080,0.008,0.613
GSM3464500,1.454,0.514,0.091,0.390,0.069,0.039,0.514,0.877,0.270,2.657,⋯,2.868,0.000,0.005,0.551,0.295,5.618,1.682,0.035,0.013,0.113
GSM3464501,1.439,0.549,0.053,0.399,0.102,0.010,0.340,1.075,0.648,4.611,⋯,7.235,0.000,0.009,0.776,1.106,11.511,1.847,0.042,0.050,0.214


In [117]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# PNH 3

In [118]:
csv_name = 'PNH_GSE80062.csv'

In [119]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [120]:
head(df)

,X1,X100,X10000,X10001,X100049587,X10005,X10006,X10007,X10008,X10009,⋯,X9985,X9986,X9987,X9988,X9989,X9990,X9991,X9993,X9994,X9997
,<int>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
PNH1_PNH,0,0,4.86132,0,0,0,54.5281,0,47.7213,0,⋯,0,0,47.3221,12.3980,85.1206,64.9785,118.7460,62.7764,0,76.5234
PNH1_WT,0,0,73.69860,0,0,0,28.8202,0,10.7316,0,⋯,0,0,68.3383,32.2032,29.4527,20.9305,52.4803,37.0561,0,37.4396
PNH2_PNH,0,0,0.00000,0,0,0,38.8762,0,0.0000,0,⋯,0,0,0.0000,0.0000,56.2583,66.8433,105.5850,41.9593,0,62.4131
PNH2_WT,0,0,0.00000,0,0,0,43.0136,0,0.0000,0,⋯,0,0,0.0000,0.0000,43.0982,61.7695,100.3670,43.8311,0,103.5360


In [121]:
colnames(df) <- sub("X", "", colnames(df))

In [122]:
library(org.Hs.eg.db)

In [123]:
keys <- keys(org.Hs.eg.db)
ref_gene <- select(org.Hs.eg.db, keys=keys, columns = c("SYMBOL", "ENTREZID"))

'select()' returned 1:1 mapping between keys and columns



In [124]:
head(ref_gene)

,ENTREZID,SYMBOL
,<chr>,<chr>
1,1,A1BG
2,2,A2M
3,3,A2MP1
4,9,NAT1
5,10,NAT2
6,11,NATP


In [125]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("ENTREZID")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="ENTREZID")

In [126]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$ENTREZID == i, 'SYMBOL'])
}

In [127]:
colnames(df) <- new_col

In [129]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [131]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [132]:
head(df)

,AKT3,ABI1,KCNE3,TANK,SRA1,TMEM170B,OST4,C17ORF107,TSTD1,PET100,⋯,CLEC2B,RBX1,DOP1B,REC8,RCE1,HNRNPDL,PPP4R1,SLC12A6,PTBP3,DGCR2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PNH1_PNH,4.86132,54.5281,47.7213,40.6043,31.0479,34.3071,179.088,0.0000,11.3232,0.0000,⋯,107.1560,153.4500,69.0983,31.3533,32.3808,42.7332,10.6406,53.5815,37.12530,226.611
PNH1_WT,73.69860,28.8202,10.7316,24.9167,15.3966,11.8819,155.451,0.0000,43.5688,0.0000,⋯,43.1518,89.6828,59.3699,30.3374,25.6837,25.9432,34.3068,36.1625,9.80725,161.636
PNH2_PNH,0.00000,38.8762,0.0000,55.8638,45.0717,0.0000,133.903,65.2098,0.0000,40.4876,⋯,67.5325,182.8790,53.1889,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000,300.054
PNH2_WT,0.00000,43.0136,0.0000,23.1306,22.1381,0.0000,147.744,36.1284,0.0000,12.2283,⋯,95.2636,117.0860,59.2075,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000,228.827


In [133]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# PEROXISOMAL 1

In [144]:
csv_name = 'PEROXISOMAL_GSE43996.csv'

In [145]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [146]:
head(df)

,DDR1.....MIR4640,RFC2,HSPA6,PAX8,GUCA1A,MIR5193.....UBA7,THRA,PTPN21,CCL5,CYP2E1,⋯,ACTB.2,ACTB.3,ACTB.4,GAPDH.3,GAPDH.4,GAPDH.5,STAT1.2,STAT1.3,STAT1.4,STAT1.5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1076221,7.0768,5.0528,4.3149,6.7425,3.9958,4.3427,5.2600,4.6215,3.4316,3.7670,⋯,12.9647,11.4188,12.5735,12.5360,12.1214,12.1436,11.1046,4.3843,5.8014,7.1729
GSM1076222,7.7073,5.5025,3.9023,6.2902,4.2264,4.1832,5.1029,4.4752,3.6022,3.8536,⋯,12.8276,11.1116,12.2317,12.5378,12.3075,12.4840,10.2477,4.1779,5.0179,6.8319
GSM1076223,7.8793,5.1808,4.3387,5.8373,3.9753,4.3371,5.3442,4.3785,3.4890,3.7592,⋯,13.0390,11.3453,12.5079,12.5174,12.2110,12.1855,11.0138,4.2171,5.4811,7.4413
GSM1076224,6.9279,5.8340,4.2696,6.5986,3.9119,4.4678,5.4809,4.3500,3.3871,3.7698,⋯,12.8536,11.2199,12.1797,12.7325,12.5639,12.7157,9.9234,4.0711,5.1117,5.7310
GSM1076225,7.8613,4.7360,4.1655,6.4940,3.9182,4.1305,5.4951,4.6855,3.4088,3.8034,⋯,13.0709,11.3927,12.6802,12.3488,11.9320,12.2582,10.4122,4.0985,5.0113,6.3496
GSM1076226,6.5881,6.3335,4.5711,6.5874,3.8701,4.8756,5.3930,4.3605,3.3986,3.7809,⋯,12.7698,11.4416,12.3531,13.0415,12.6648,12.7245,10.0229,3.7954,5.1809,5.7449


In [158]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [159]:
# Remove double names genes
colnames(df) <- gsub("\\...*", "", colnames(df))

In [160]:
head(df)

,DDR1,RFC2,HSPA6,PAX8,GUCA1A,MIR5193,THRA,PTPN21,CCL5,CYP2E1,⋯,ACTB,ACTB.1,ACTB.2,GAPDH,GAPDH.1,GAPDH.2,STAT1,STAT1.1,STAT1.2,STAT1.3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1076221,7.0768,5.0528,4.3149,6.7425,3.9958,4.3427,5.2600,4.6215,3.4316,3.7670,⋯,12.9647,11.4188,12.5735,12.5360,12.1214,12.1436,11.1046,4.3843,5.8014,7.1729
GSM1076222,7.7073,5.5025,3.9023,6.2902,4.2264,4.1832,5.1029,4.4752,3.6022,3.8536,⋯,12.8276,11.1116,12.2317,12.5378,12.3075,12.4840,10.2477,4.1779,5.0179,6.8319
GSM1076223,7.8793,5.1808,4.3387,5.8373,3.9753,4.3371,5.3442,4.3785,3.4890,3.7592,⋯,13.0390,11.3453,12.5079,12.5174,12.2110,12.1855,11.0138,4.2171,5.4811,7.4413
GSM1076224,6.9279,5.8340,4.2696,6.5986,3.9119,4.4678,5.4809,4.3500,3.3871,3.7698,⋯,12.8536,11.2199,12.1797,12.7325,12.5639,12.7157,9.9234,4.0711,5.1117,5.7310
GSM1076225,7.8613,4.7360,4.1655,6.4940,3.9182,4.1305,5.4951,4.6855,3.4088,3.8034,⋯,13.0709,11.3927,12.6802,12.3488,11.9320,12.2582,10.4122,4.0985,5.0113,6.3496
GSM1076226,6.5881,6.3335,4.5711,6.5874,3.8701,4.8756,5.3930,4.3605,3.3986,3.7809,⋯,12.7698,11.4416,12.3531,13.0415,12.6648,12.7245,10.0229,3.7954,5.1809,5.7449


In [161]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# PEROXISOMAL 2

In [162]:
csv_name = 'PEROXISOMAL_GSE69066.csv'

In [163]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [164]:
head(df)

,DDR1.....MIR4640,RFC2,HSPA6,PAX8,GUCA1A,MIR5193.....UBA7,THRA,PTPN21,CCL5,CYP2E1,⋯,ACTB.2,ACTB.3,ACTB.4,GAPDH.3,GAPDH.4,GAPDH.5,STAT1.2,STAT1.3,STAT1.4,STAT1.5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1691855,6.153530,2.114966,0.8662884,3.290827,0.5929813,0.8002804,0.6966309,1.333083,0.3372364,0.7694430,⋯,10.90291,9.060856,10.501245,10.66562,10.484826,10.66460,5.799227,0.5929813,2.115300,2.293285
GSM1691856,6.605481,2.114966,1.3830027,3.636813,0.5929813,1.0350757,1.1332877,1.569880,0.3372364,0.5929813,⋯,10.39898,7.365249,9.394796,10.42223,9.805768,10.12718,5.272491,0.5929813,1.985572,1.695409


In [165]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [166]:
# Remove double names genes
colnames(df) <- gsub("\\...*", "", colnames(df))

In [167]:
head(df)

,DDR1,RFC2,HSPA6,PAX8,GUCA1A,MIR5193,THRA,PTPN21,CCL5,CYP2E1,⋯,ACTB,ACTB.1,ACTB.2,GAPDH,GAPDH.1,GAPDH.2,STAT1,STAT1.1,STAT1.2,STAT1.3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1691855,6.153530,2.114966,0.8662884,3.290827,0.5929813,0.8002804,0.6966309,1.333083,0.3372364,0.7694430,⋯,10.90291,9.060856,10.501245,10.66562,10.484826,10.66460,5.799227,0.5929813,2.115300,2.293285
GSM1691856,6.605481,2.114966,1.3830027,3.636813,0.5929813,1.0350757,1.1332877,1.569880,0.3372364,0.5929813,⋯,10.39898,7.365249,9.394796,10.42223,9.805768,10.12718,5.272491,0.5929813,1.985572,1.695409


In [168]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# GAUCHER 1

In [169]:
csv_name = 'GAUCHER_GSE124283.csv'

In [170]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [171]:
head(df)

,X7A5,A1BG,A1CF,A26C3,A2BP1,A2LD1,A2M,A2ML1,A3GALT2,A4GALT,⋯,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM3526979,77.90689,77.91269,76.00592,75.28105,74.41833,89.92991,83.31320,71.61835,76.07062,173.85630,⋯,110.37890,94.28487,77.18758,99.54223,81.95280,78.62696,405.7758,1214.3800,260.9414,203.0276
GSM3526980,75.86201,80.11732,74.23648,72.94248,75.92688,92.90813,71.50749,72.88329,72.94296,253.54580,⋯,93.42171,81.22705,79.11552,89.55640,85.88889,76.47934,351.6067,835.2040,190.0189,196.1484
GSM3526988,72.46059,75.50118,72.61402,74.34894,70.92567,83.59776,71.76194,70.64061,74.49350,97.74953,⋯,154.72310,122.07520,74.51260,80.50256,75.28564,73.82278,170.6696,440.1375,127.0901,190.6778
GSM3527004,82.72894,80.58268,82.17849,79.12257,78.48637,120.71830,190.78470,80.77851,77.38298,168.05590,⋯,155.56360,97.40102,81.52473,102.42350,93.35011,78.53728,718.7820,1222.2520,288.9493,388.2021
GSM3527007,77.87585,85.32943,78.29251,78.42223,77.56104,111.99370,77.09916,77.24261,80.02428,149.84040,⋯,228.03460,151.03060,80.03234,90.15014,90.44879,78.01225,510.9697,1037.1090,233.7106,404.4689
GSM3527008,82.67387,82.40833,80.29066,78.62093,75.76055,111.41830,99.75630,73.16680,74.18369,123.66790,⋯,150.79300,88.90922,79.73350,102.26050,88.67117,74.79890,551.6744,1240.8010,197.4138,340.2071


In [172]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [173]:
head(df)

,X7A5,A1BG,A1CF,A26C3,A2BP1,A2LD1,A2M,A2ML1,A3GALT2,A4GALT,⋯,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM3526979,77.90689,77.91269,76.00592,75.28105,74.41833,89.92991,83.31320,71.61835,76.07062,173.85630,⋯,110.37890,94.28487,77.18758,99.54223,81.95280,78.62696,405.7758,1214.3800,260.9414,203.0276
GSM3526980,75.86201,80.11732,74.23648,72.94248,75.92688,92.90813,71.50749,72.88329,72.94296,253.54580,⋯,93.42171,81.22705,79.11552,89.55640,85.88889,76.47934,351.6067,835.2040,190.0189,196.1484
GSM3526988,72.46059,75.50118,72.61402,74.34894,70.92567,83.59776,71.76194,70.64061,74.49350,97.74953,⋯,154.72310,122.07520,74.51260,80.50256,75.28564,73.82278,170.6696,440.1375,127.0901,190.6778
GSM3527004,82.72894,80.58268,82.17849,79.12257,78.48637,120.71830,190.78470,80.77851,77.38298,168.05590,⋯,155.56360,97.40102,81.52473,102.42350,93.35011,78.53728,718.7820,1222.2520,288.9493,388.2021
GSM3527007,77.87585,85.32943,78.29251,78.42223,77.56104,111.99370,77.09916,77.24261,80.02428,149.84040,⋯,228.03460,151.03060,80.03234,90.15014,90.44879,78.01225,510.9697,1037.1090,233.7106,404.4689
GSM3527008,82.67387,82.40833,80.29066,78.62093,75.76055,111.41830,99.75630,73.16680,74.18369,123.66790,⋯,150.79300,88.90922,79.73350,102.26050,88.67117,74.79890,551.6744,1240.8010,197.4138,340.2071


In [174]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# GAUCHER 2

In [175]:
csv_name = 'GAUCHER_GSE13675.csv'

In [176]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [177]:
head(df)

,ACAT1,PIGA,PRPS1L1,PTPN12,SDHB,IL1RL1,RBL1,PREPL,FGF9,PRDX6,⋯,DZIP3,BIRC4.1,VKORC1L1.1,PLEKHA5.1,CNOT6,PLXNA1.2,RBM26.1,HNRNPR.1,ANAPC1.1,GPATCH8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<lgl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM343899,307.5,933.5,275.000,689.25,6628.50,108.25,158.50,445.75,NA,19886.25,⋯,NA,3408.875,122.250,128.75,132,70.5,211.25,115.75,125.25,99.75
GSM343901,304.0,1123.5,269.125,690.50,7257.75,164.75,158.75,388.75,NA,18588.50,⋯,NA,3060.500,95.125,90.00,160,21.0,243.75,127.00,161.25,91.00


In [179]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [180]:
head(df)

,ACAT1,PIGA,PRPS1L1,PTPN12,SDHB,IL1RL1,RBL1,PREPL,PRDX6,CCKAR,⋯,SEC24A.1,BIRC4.1,VKORC1L1.1,PLEKHA5.1,CNOT6,PLXNA1.2,RBM26.1,HNRNPR.1,ANAPC1.1,GPATCH8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM343899,307.5,933.5,275.000,689.25,6628.50,108.25,158.50,445.75,19886.25,62.75,⋯,107.25,3408.875,122.250,128.75,132,70.5,211.25,115.75,125.25,99.75
GSM343901,304.0,1123.5,269.125,690.50,7257.75,164.75,158.75,388.75,18588.50,70.75,⋯,80.25,3060.500,95.125,90.00,160,21.0,243.75,127.00,161.25,91.00


In [181]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# GAUCHER 3

In [182]:
csv_name = 'GAUCHER_GSE21899.csv'

In [183]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [184]:
head(df)

,ACAT1,PIGA,PRPS1L1,PTPN12,SDHB,IL1RL1,RBL1,PREPL,PRDX6,CCKAR,⋯,SEC24A.1,BIRC4.1,VKORC1L1.1,PLEKHA5.1,CNOT6,PLXNA1.2,RBM26.1,HNRNPR.1,ANAPC1.1,GPATCH8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM343899,307.5,933.5,275.000,689.25,6628.50,108.25,158.50,445.75,19886.25,62.75,⋯,107.25,3408.875,122.250,128.75,132,70.5,211.25,115.75,125.25,99.75
GSM343901,304.0,1123.5,269.125,690.50,7257.75,164.75,158.75,388.75,18588.50,70.75,⋯,80.25,3060.500,95.125,90.00,160,21.0,243.75,127.00,161.25,91.00


In [185]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [186]:
head(df)

,ACAT1,PIGA,PRPS1L1,PTPN12,SDHB,IL1RL1,RBL1,PREPL,PRDX6,CCKAR,⋯,SEC24A.1,BIRC4.1,VKORC1L1.1,PLEKHA5.1,CNOT6,PLXNA1.2,RBM26.1,HNRNPR.1,ANAPC1.1,GPATCH8
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM343899,307.5,933.5,275.000,689.25,6628.50,108.25,158.50,445.75,19886.25,62.75,⋯,107.25,3408.875,122.250,128.75,132,70.5,211.25,115.75,125.25,99.75
GSM343901,304.0,1123.5,269.125,690.50,7257.75,164.75,158.75,388.75,18588.50,70.75,⋯,80.25,3060.500,95.125,90.00,160,21.0,243.75,127.00,161.25,91.00


In [187]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# GAUCHER 4

In [259]:
csv_name = 'GAUCHER_GSE41243.csv'

In [260]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [261]:
head(df)

,X7892501,X7892502,X7892503,X7892504,X7892505,X7892506,X7892507,X7892508,X7892509,X7892510,⋯,X8180408,X8180409,X8180410,X8180411,X8180413,X8180414,X8180415,X8180416,X8180417,X8180418
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1012132,2.622779,3.71082,5.17084,9.369058,3.376183,3.827842,6.516457,4.931786,10.55233,3.403059,⋯,9.018806,10.69688,11.40912,6.130435,7.879939,5.742754,7.893209,5.229703,8.427243,9.467293


In [262]:
colnames(df) <- sub("X", "", colnames(df))

In [263]:
library(hugene10sttranscriptcluster.db)
# Get gene symbol
x <- hugene10sttranscriptclusterSYMBOL
# Get the probe identifiers that are mapped to a gene name
mapped_probes <- mappedkeys(x)
# Convert to a list
xx <- as.list(x[mapped_probes])

In [264]:
# Transform matrix in a table
ref_gene <- do.call(rbind, xx)
ref_gene <- as.data.frame(ref_gene)
# Add column to get a reference between old and new gene
ref_gene$old_gene <- rownames(ref_gene)
colnames(ref_gene) <- c("new_gene", "old_gene")

In [267]:
head(ref_gene)

,new_gene,old_gene
,<chr>,<chr>
7896746,MTND1P23,7896746
7896754,SEPTIN7P13,7896754
7896759,LINC01128,7896759
7896761,SAMD11,7896761
7896779,KLHL17,7896779
7896798,PLEKHN1,7896798
7896817,ISG15,7896817
7896822,AGRN,7896822
7896859,MIR200B,7896859


In [252]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("old_gene")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="old_gene")

In [268]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$old_gene == i, 'new_gene'])
}

In [269]:
colnames(df) <- new_col

In [270]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [271]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [272]:
head(df)

,MTND1P23,LINC01128,SAMD11,KLHL17,PLEKHN1,ISG15,AGRN,MIR200B,MIR200A,MIR429,⋯,SPRR2G,NPM1.1,NPM1.2,EIF3F.1,ELMO1.1,SLC25A51.1,OR3A2.1,METTL2B.1,CHTF8,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM1012132,2.622779,3.71082,5.17084,9.369058,3.376183,3.827842,6.516457,4.931786,10.55233,3.403059,⋯,6.613996,2.973034,7.233088,5.50279,6.918239,4.234672,6.520126,6.587063,8.955726,10.46856


In [273]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# NCL

In [4]:
csv_name = 'eGSE171276m.csv'

In [10]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), sep="", header=TRUE)

In [16]:
head(df)

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,⋯,BX005132.2,AL645728.2,PDE11A.1,AL163152.1,AL139254.3,AL591163.1,AL589702.1,AC068587.10,AL034417.4,AL513220.1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GRN.Rep0,4.567620,1e-05,17.72261,1.95574,1.445130,161.428,0.1303780,47.90601,105.08101,11.24911,⋯,1e-05,0.412272,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.298582,1e-05
GRN.Rep1,4.175720,1e-05,20.55151,1.65214,1.928600,167.051,0.2493420,50.68341,100.08501,10.26511,⋯,1e-05,0.000010,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.439470,1e-05
GRN.Rep2,4.056860,1e-05,20.37221,1.60467,3.689060,153.524,0.3744310,50.55921,114.85701,11.74231,⋯,1e-05,0.321201,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.277072,1e-05
MeCP2.Rep0,0.670684,1e-05,21.05121,1.35741,0.512520,235.849,0.2991940,71.84811,11.57551,2.91352,⋯,1e-05,0.000010,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.666880,1e-05
SNCA.Rep0,4.691710,1e-05,21.24151,1.83863,1.690950,158.569,0.1790100,49.03211,117.49601,10.60711,⋯,1e-05,1.564350,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.267216,1e-05
AHN.Rep0,8.377920,1e-05,25.51011,1.23766,0.879958,189.702,0.0887336,65.85901,37.08351,5.65463,⋯,1e-05,0.407233,1.258147e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.461774,1e-05


In [12]:
rownames(df) <- df$X

In [15]:
df <- df[,2:ncol(df)]

In [17]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [18]:
head(df)

,TSPAN6,TNMD,DPM1,SCYL3,C1ORF112,FGR,CFH,FUCA2,GCLC,NFYA,⋯,BX005132.2,AL645728.2,PDE11A.1,AL163152.1,AL139254.3,AL591163.1,AL589702.1,AC068587.10,AL034417.4,AL513220.1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GRN.Rep0,4.567620,1e-05,17.72261,1.95574,1.445130,161.428,0.1303780,47.90601,105.08101,11.24911,⋯,1e-05,0.412272,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.298582,1e-05
GRN.Rep1,4.175720,1e-05,20.55151,1.65214,1.928600,167.051,0.2493420,50.68341,100.08501,10.26511,⋯,1e-05,0.000010,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.439470,1e-05
GRN.Rep2,4.056860,1e-05,20.37221,1.60467,3.689060,153.524,0.3744310,50.55921,114.85701,11.74231,⋯,1e-05,0.321201,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.277072,1e-05
MeCP2.Rep0,0.670684,1e-05,21.05121,1.35741,0.512520,235.849,0.2991940,71.84811,11.57551,2.91352,⋯,1e-05,0.000010,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.666880,1e-05
SNCA.Rep0,4.691710,1e-05,21.24151,1.83863,1.690950,158.569,0.1790100,49.03211,117.49601,10.60711,⋯,1e-05,1.564350,1.000000e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.267216,1e-05
AHN.Rep0,8.377920,1e-05,25.51011,1.23766,0.879958,189.702,0.0887336,65.85901,37.08351,5.65463,⋯,1e-05,0.407233,1.258147e-05,1e-05,1e-05,1e-05,1e-05,1e-05,0.461774,1e-05


In [19]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# NCL 2

In [20]:
csv_name = 'eGSE30369m.csv'

In [21]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), sep="", header=TRUE)

In [22]:
head(df)

,X,ILMN_1651209,ILMN_1651217,ILMN_1651228,ILMN_1651229,ILMN_1651234,ILMN_1651235,ILMN_1651236,ILMN_1651237,ILMN_1651238,⋯,ILMN_1815885,ILMN_1815890,ILMN_1815908,ILMN_1815923,ILMN_1815924,ILMN_1815933,ILMN_1815937,ILMN_1815938,ILMN_1815941,ILMN_1815951
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,GSM753551,6.441433,6.292399,13.43724,7.988214,5.696954,6.035412,5.935416,6.093311,6.048240,⋯,6.398945,8.053059,6.554411,5.828407,9.418736,7.057031,5.944536,5.820384,8.478805,7.652425
2,GSM753552,6.501252,6.199564,13.87455,7.613662,5.792640,6.021823,6.047903,5.857664,5.942314,⋯,6.513539,8.014108,6.380647,5.810531,8.893691,7.203604,6.020227,5.740604,8.816835,7.723623
3,GSM753553,6.217822,5.942144,13.99501,7.578761,5.724316,6.011809,5.936246,5.877185,5.807121,⋯,6.435321,7.409345,6.182712,5.661384,9.257995,7.027332,6.050437,5.981099,9.142315,7.678386
4,GSM753554,6.100076,6.019551,13.84246,7.689085,5.762888,6.097306,6.033793,5.800126,5.942409,⋯,6.531783,7.560220,6.421435,5.770118,9.384104,7.323473,6.068272,6.071177,8.226703,7.646915


In [23]:
rownames(df) <- df$X

In [24]:
df <- df[,2:ncol(df)]

In [25]:
library(illuminaHumanv2.db)
# Get gene symbol
x <- illuminaHumanv2SYMBOL
# Get the probe identifiers that are mapped to a gene name
mapped_probes <- mappedkeys(x)
# Convert to a list
xx <- as.list(x[mapped_probes])

Caricamento del pacchetto richiesto: org.Hs.eg.db







In [26]:
# Transform matrix in a table
ref_gene <- do.call(rbind, xx)
ref_gene <- as.data.frame(ref_gene)
# Add column to get a reference between old and new gene
ref_gene$old_gene <- rownames(ref_gene)
colnames(ref_gene) <- c("new_gene", "old_gene")

In [27]:
head(ref_gene)

,new_gene,old_gene
,<chr>,<chr>
ILMN_1343289,RPS9,ILMN_1343289
ILMN_1343290,UBC,ILMN_1343290
ILMN_1343291,EEF1A1,ILMN_1343291
ILMN_1343292,TUBB2A,ILMN_1343292
ILMN_1343293,TXN,ILMN_1343293
ILMN_1343294,ACTB,ILMN_1343294


In [28]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("old_gene")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="old_gene")

In [29]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$old_gene == i, 'new_gene'])
}

In [30]:
colnames(df) <- new_col

In [32]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [33]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [34]:
head(df)

,SLC35E2A,PDCD1LG2,RPS28,IPO13,SYT14,AFAP1,CDT1,TRPV1,LPP,UGP2,⋯,IL12RB1.2,GYPE.2,MIEF2.1,NUP107,MRM2,TMEM184A,TRPM3.1,SMAD7,PCYOX1L,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM753551,6.441433,6.292399,13.43724,7.988214,5.696954,6.035412,5.935416,6.093311,6.048240,10.88280,⋯,8.881951,5.742600,9.761695,8.399498,9.348179,5.823650,5.760643,7.008496,9.556403,5.982533
GSM753552,6.501252,6.199564,13.87455,7.613662,5.792640,6.021823,6.047903,5.857664,5.942314,10.77981,⋯,8.888432,5.680141,9.329160,8.636070,9.081144,5.635906,5.774344,6.962168,9.510372,6.045377
GSM753553,6.217822,5.942144,13.99501,7.578761,5.724316,6.011809,5.936246,5.877185,5.807121,10.93185,⋯,8.667122,5.818894,9.580230,8.689073,8.937797,5.602713,5.672968,6.705277,9.727296,6.013081
GSM753554,6.100076,6.019551,13.84246,7.689085,5.762888,6.097306,6.033793,5.800126,5.942409,10.80619,⋯,8.609744,5.759345,9.962157,8.409808,9.151297,5.802742,5.707643,6.989262,9.684138,6.185584


In [35]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# NCL 3

In [73]:
csv_name = 'eGSE22225m.csv'

In [76]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), sep="", header=TRUE)

In [81]:
head(df)

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,⋯,AFFX.r2.Ec.bioD.3_at,AFFX.r2.Ec.bioD.5_at,AFFX.r2.P1.cre.3_at,AFFX.r2.P1.cre.5_at,AFFX.ThrX.3_at,AFFX.ThrX.5_at,AFFX.ThrX.M_at,AFFX.TrpnX.3_at,AFFX.TrpnX.5_at,AFFX.TrpnX.M_at
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM553488,402.316,49.9719,316.742,137.057,2.00192,2382.58,56.0270,1.74040,12702.10,14.1855,⋯,6595.42,5951.66,18369.3,14085.9,1.40761,2.78140,11.393000,0.781948,1.42560,1.057220
GSM553489,280.852,141.7080,229.717,198.294,3.80613,1782.65,38.5179,3.17614,12908.10,18.7509,⋯,6258.48,5641.96,17875.1,13297.1,4.35850,1.11712,1.255180,1.758010,2.75364,2.113560
GSM553490,233.065,55.2751,237.900,176.998,5.74062,2029.92,41.4362,1.90828,12115.70,22.6535,⋯,6325.02,5557.57,17395.3,13134.4,1.62565,2.22278,3.416090,2.437500,18.24970,6.341010
GSM553491,322.712,121.6030,133.198,111.450,2.03515,1474.99,20.6418,3.62619,6868.38,20.2586,⋯,4393.02,4077.11,15164.8,10507.1,1.49038,2.48639,0.618481,6.497000,3.27684,3.018970
GSM553492,356.386,100.0110,426.484,124.452,8.60303,2433.58,29.9426,1.41034,8207.56,34.2108,⋯,5734.50,4934.68,14623.1,11621.4,2.27668,1.78621,2.180720,6.339210,9.84354,0.959524
GSM553493,399.190,81.0371,240.577,152.464,1.49584,2591.47,66.9889,4.67163,8641.03,24.9140,⋯,5856.73,4993.38,15664.8,12378.0,1.71180,2.77251,0.641685,0.920228,6.35599,2.201160


In [78]:
rownames(df) <- df$X

In [79]:
df <- df[,2:ncol(df)]

In [80]:
colnames(df) <- sub("^X", "", colnames(df))

In [82]:
library(hgu133plus2.db)
# Get gene symbol
x <- hgu133plus2SYMBOL
# Get the probe identifiers that are mapped to a gene name
mapped_probes <- mappedkeys(x)
# Convert to a list
xx <- as.list(x[mapped_probes])

In [83]:
# Transform matrix in a table
ref_gene <- do.call(rbind, xx)
ref_gene <- as.data.frame(ref_gene)
# Add column to get a reference between old and new gene
ref_gene$old_gene <- rownames(ref_gene)
colnames(ref_gene) <- c("new_gene", "old_gene")

In [86]:
head(ref_gene)

,old_gene,new_gene
,<chr>,<chr>
1,1007_s_at,DDR1
2,1053_at,RFC2
3,117_at,HSPA6
4,121_at,PAX8
5,1255_g_at,GUCA1A
6,1294_at,UBA7


In [85]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("old_gene")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="old_gene")

In [87]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$old_gene == i, 'new_gene'])
}

In [88]:
colnames(df) <- new_col

In [90]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [91]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [92]:
head(df)

,DDR1,RFC2,HSPA6,PAX8,GUCA1A,UBA7,THRA,PTPN21,CCL5,CYP2E1,⋯,LRTM1.1,DGCR8.3,EXOSC4.2,EXOSC4.3,EHBP1L1.3,MEX3D.2,EPS8L1.4,BCAN.2,DCAF15.3,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM553488,402.316,49.9719,316.742,137.057,2.00192,2382.58,56.0270,1.74040,12702.10,14.1855,⋯,137.7770,62.17850,8.37484,1.220240,222.116,46.9557,2.70320,4.26834,12.6071,1.79327
GSM553489,280.852,141.7080,229.717,198.294,3.80613,1782.65,38.5179,3.17614,12908.10,18.7509,⋯,61.4688,22.91660,57.17760,4.201630,133.852,50.3944,1.43971,9.80503,46.7709,8.04823
GSM553490,233.065,55.2751,237.900,176.998,5.74062,2029.92,41.4362,1.90828,12115.70,22.6535,⋯,54.1747,30.97150,38.33630,0.563291,189.800,64.4948,3.71684,14.04010,28.2501,2.80511
GSM553491,322.712,121.6030,133.198,111.450,2.03515,1474.99,20.6418,3.62619,6868.38,20.2586,⋯,13.6870,2.40774,127.00000,1.735340,161.193,62.5642,4.02385,2.79884,33.4408,3.29151
GSM553492,356.386,100.0110,426.484,124.452,8.60303,2433.58,29.9426,1.41034,8207.56,34.2108,⋯,24.5837,34.50810,74.38060,2.071230,214.288,64.3161,12.08490,2.03465,51.4488,4.69052
GSM553493,399.190,81.0371,240.577,152.464,1.49584,2591.47,66.9889,4.67163,8641.03,24.9140,⋯,62.3704,9.22732,31.22420,0.980133,169.738,47.7095,3.23263,7.15798,62.4847,7.55601


In [93]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# NALD 1

In [94]:
csv_name = 'eGSE117647m.csv'

In [95]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), sep="", header=TRUE)

In [100]:
head(df)

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,⋯,AFFX.r2.Hs28SrRNA.5_at,AFFX.r2.Hs28SrRNA.M_at,AFFX.r2.P1.cre.3_at,AFFX.r2.P1.cre.5_at,AFFX.ThrX.3_at,AFFX.ThrX.5_at,AFFX.ThrX.M_at,AFFX.TrpnX.3_at,AFFX.TrpnX.5_at,AFFX.TrpnX.M_at
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM3305543,10.5602,4.88739,6.37506,8.21095,3.85878,6.82803,5.80560,4.56967,3.83793,4.03723,⋯,6.17693,6.25184,12.7736,12.4525,4.80926,4.05225,3.56501,3.35627,3.73266,3.60671
GSM3305544,10.4210,4.69174,6.36858,8.16335,3.90241,6.77299,5.98227,4.56041,3.89118,4.06312,⋯,5.77611,6.31226,12.7681,12.4877,4.83666,4.08639,3.51865,3.44515,3.69921,3.69556
GSM3305545,10.5141,4.67749,6.36301,8.12071,3.89135,6.98857,5.82710,4.70470,3.98109,4.17354,⋯,5.59198,6.29947,12.5715,12.2950,4.85267,4.16020,3.58586,3.44989,3.79205,3.67558
GSM3305546,10.3080,4.71070,6.14188,7.89710,3.78553,6.71299,5.84008,4.53418,3.85541,4.08813,⋯,6.36244,6.84612,12.7375,12.6246,4.65517,4.03839,3.44941,3.39253,3.65439,3.53906
GSM3305547,10.3174,4.67528,6.41441,8.05305,3.90189,6.72251,5.87682,4.59815,3.91390,4.07607,⋯,5.45843,6.11015,12.5979,12.2991,4.83351,4.09642,3.53042,3.45339,3.72413,3.68635
GSM3305548,10.3783,4.55753,6.75081,7.99613,3.88854,6.75443,5.95493,4.47163,3.86128,3.97068,⋯,6.30147,6.98928,12.6244,12.3425,4.77618,4.05246,3.53213,3.38350,3.62129,3.55534


In [97]:
rownames(df) <- df$X

In [98]:
df <- df[,2:ncol(df)]

In [99]:
colnames(df) <- sub("^X", "", colnames(df))

In [101]:
library(hgu133a.db)
# Get gene symbol
x <- hgu133aSYMBOL
# Get the probe identifiers that are mapped to a gene name
mapped_probes <- mappedkeys(x)
# Convert to a list
xx <- as.list(x[mapped_probes])

In [102]:
# Transform matrix in a table
ref_gene <- do.call(rbind, xx)
ref_gene <- as.data.frame(ref_gene)
# Add column to get a reference between old and new gene
ref_gene$old_gene <- rownames(ref_gene)
colnames(ref_gene) <- c("new_gene", "old_gene")

In [105]:
head(ref_gene)

,old_gene,new_gene
,<chr>,<chr>
1,1007_s_at,DDR1
2,1053_at,RFC2
3,117_at,HSPA6
4,121_at,PAX8
5,1255_g_at,GUCA1A
6,1294_at,UBA7


In [104]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("old_gene")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="old_gene")

In [106]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$old_gene == i, 'new_gene'])
}

In [107]:
colnames(df) <- new_col

In [108]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [109]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [110]:
head(df)

,DDR1,RFC2,HSPA6,PAX8,GUCA1A,UBA7,THRA,PTPN21,CCL5,CYP2E1,⋯,LRTM1.1,DGCR8.3,EXOSC4.2,EXOSC4.3,EHBP1L1.1,MEX3D.1,EPS8L1.4,BCAN.2,DCAF15.3,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM3305543,10.5602,4.88739,6.37506,8.21095,3.85878,6.82803,5.80560,4.56967,3.83793,4.03723,⋯,6.53580,5.52100,5.90202,5.68591,4.59300,4.34913,4.26716,5.33330,5.01056,5.08519
GSM3305544,10.4210,4.69174,6.36858,8.16335,3.90241,6.77299,5.98227,4.56041,3.89118,4.06312,⋯,6.81556,5.67873,5.69920,5.95416,4.53068,4.42343,4.41385,5.58897,4.91408,4.94859
GSM3305545,10.5141,4.67749,6.36301,8.12071,3.89135,6.98857,5.82710,4.70470,3.98109,4.17354,⋯,6.82930,5.72211,5.84393,5.96313,4.50824,4.32699,4.50664,5.59945,5.12298,5.16015
GSM3305546,10.3080,4.71070,6.14188,7.89710,3.78553,6.71299,5.84008,4.53418,3.85541,4.08813,⋯,6.81599,5.40186,5.26108,5.58416,4.53961,4.22180,4.31490,5.24472,4.77130,4.88011
GSM3305547,10.3174,4.67528,6.41441,8.05305,3.90189,6.72251,5.87682,4.59815,3.91390,4.07607,⋯,6.86601,5.80670,5.66685,5.73578,4.44977,4.44079,4.32020,5.49581,5.02155,5.21626
GSM3305548,10.3783,4.55753,6.75081,7.99613,3.88854,6.75443,5.95493,4.47163,3.86128,3.97068,⋯,6.77384,5.48603,5.78242,5.98683,4.48202,4.25062,4.33747,5.31338,4.86964,5.13845


In [111]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# NALD 2

In [115]:
csv_name = 'eGSE85804m.csv'

In [116]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), sep="", header=TRUE)

In [117]:
head(df)

,X,ILMN_1343291,ILMN_1343295,ILMN_1651199,ILMN_1651209,ILMN_1651210,ILMN_1651221,ILMN_1651228,ILMN_1651229,ILMN_1651230,⋯,ILMN_3311145,ILMN_3311150,ILMN_3311155,ILMN_3311160,ILMN_3311165,ILMN_3311170,ILMN_3311175,ILMN_3311180,ILMN_3311185,ILMN_3311190
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,GSM2284671,15.83386,14.54472,6.281292,6.619897,6.413851,6.638063,10.81211,8.828693,6.476976,⋯,6.801084,6.451900,6.842599,6.645013,7.031729,6.623841,6.707154,6.819731,6.825700,7.131148
2,GSM2284672,15.83386,14.75326,6.561281,6.643221,6.403661,6.754368,11.11089,9.102738,6.536011,⋯,6.623955,6.711346,6.760629,6.515061,7.211130,6.608122,6.829850,6.835666,6.534389,6.969426
3,GSM2284673,15.79367,14.85705,6.480725,6.833132,6.507696,6.723469,10.83922,9.340438,6.359756,⋯,6.674612,6.710991,6.811371,6.423476,7.086390,6.588900,6.810340,6.850024,6.678133,7.153313
4,GSM2284677,15.71969,14.70113,6.536609,6.667342,6.455909,6.968357,11.63309,9.426174,6.504793,⋯,6.722703,6.581198,6.721649,6.341482,7.128031,6.505155,6.766391,7.025292,6.722994,7.244974
5,GSM2284678,15.77584,14.76104,6.397413,6.507473,6.537430,6.732759,11.48547,9.175696,6.505945,⋯,6.807464,6.773498,6.869154,6.483392,7.058450,6.551044,6.805510,6.980071,6.674849,7.177885
6,GSM2284679,15.77584,14.82463,6.159434,6.703762,6.462397,6.982827,11.65260,9.766835,6.512670,⋯,6.808123,6.485108,6.683724,6.406548,7.104918,6.707592,6.679405,6.860728,6.484390,7.088159


In [118]:
rownames(df) <- df$X

In [119]:
df <- df[,2:ncol(df)]

In [120]:
library(illuminaHumanv4.db)
# Get gene symbol
x <- illuminaHumanv4SYMBOL
# Get the probe identifiers that are mapped to a gene name
mapped_probes <- mappedkeys(x)
# Convert to a list
xx <- as.list(x[mapped_probes])

In [121]:
# Transform matrix in a table
ref_gene <- do.call(rbind, xx)
ref_gene <- as.data.frame(ref_gene)
# Add column to get a reference between old and new gene
ref_gene$old_gene <- rownames(ref_gene)
colnames(ref_gene) <- c("new_gene", "old_gene")

In [124]:
head(ref_gene)

,old_gene,new_gene
,<chr>,<chr>
1,ILMN_1343291,EEF1A1
2,ILMN_1343295,GAPDH
3,ILMN_1651209,SLC35E2A
4,ILMN_1651221,EFCAB1
5,ILMN_1651228,RPS28
6,ILMN_1651229,IPO13


In [123]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("old_gene")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="old_gene")

In [125]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$old_gene == i, 'new_gene'])
}

In [126]:
colnames(df) <- new_col

In [128]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [129]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [130]:
head(df)

,EEF1A1,GAPDH,SLC35E2A,EFCAB1,RPS28,IPO13,PRSS41,FAM86FP,AFAP1,CDT1,⋯,MIR335,MIR663B,MIR548P,MIR320C1,MIR1255B2,LINC00293.1,LINC00293.2,LINC00173.1,LINC00173.2,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM2284671,15.83386,14.54472,6.281292,6.619897,6.413851,6.638063,10.81211,8.828693,6.476976,7.023736,⋯,6.236264,9.298238,6.253194,6.560741,8.888861,6.772176,6.740707,6.354117,6.478599,9.922648
GSM2284672,15.83386,14.75326,6.561281,6.643221,6.403661,6.754368,11.11089,9.102738,6.536011,7.225084,⋯,6.447237,9.413363,6.687802,6.174438,8.838217,6.842338,7.453013,6.523148,6.164814,9.806783
GSM2284673,15.79367,14.85705,6.480725,6.833132,6.507696,6.723469,10.83922,9.340438,6.359756,7.096716,⋯,6.297002,9.081304,6.522567,6.380331,8.737239,6.676393,7.194861,6.394657,6.275892,9.471546
GSM2284677,15.71969,14.70113,6.536609,6.667342,6.455909,6.968357,11.63309,9.426174,6.504793,6.811274,⋯,6.338141,9.836738,6.539993,6.104720,9.025804,6.633863,7.569025,6.664560,6.215367,9.756255
GSM2284678,15.77584,14.76104,6.397413,6.507473,6.537430,6.732759,11.48547,9.175696,6.505945,7.016635,⋯,6.388176,9.666861,6.629297,6.368728,9.086856,6.704024,7.391966,6.589769,6.193584,9.902244
GSM2284679,15.77584,14.82463,6.159434,6.703762,6.462397,6.982827,11.65260,9.766835,6.512670,7.171394,⋯,6.437337,8.713492,6.687672,6.320088,9.018861,6.651407,7.176205,6.539830,6.311227,8.475927


In [131]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# NALD 3

In [132]:
csv_name = 'eGSE34308m.csv'

In [133]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), sep="", header=TRUE)

In [138]:
head(df)

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,⋯,AFFX.r2.Ec.bioD.3_at,AFFX.r2.Ec.bioD.5_at,AFFX.r2.P1.cre.3_at,AFFX.r2.P1.cre.5_at,AFFX.ThrX.3_at,AFFX.ThrX.5_at,AFFX.ThrX.M_at,AFFX.TrpnX.3_at,AFFX.TrpnX.5_at,AFFX.TrpnX.M_at
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM847083,6.856013,4.872198,3.070721,5.949972,2.563389,6.549806,3.887666,4.335238,2.631582,2.742217,⋯,10.84725,10.401905,12.48716,12.33186,4.404147,3.188694,3.676707,2.623115,2.950158,2.712637
GSM847084,6.373692,5.160413,3.086563,6.187076,2.833957,6.397387,4.040880,4.335334,2.513761,2.724881,⋯,11.38098,10.724874,12.80207,12.34969,4.615457,3.495858,3.562872,2.672477,3.116305,2.710284
GSM847085,6.562429,5.359019,3.207338,6.280731,2.577091,5.999377,4.046283,4.446986,2.730223,2.905883,⋯,10.29835,9.764847,12.25980,11.95660,6.626405,4.127738,4.876908,2.591166,2.885586,2.960692
GSM847086,6.934789,6.031420,3.232655,6.416759,2.684375,5.246129,4.077676,4.198332,2.495319,2.808868,⋯,10.68187,10.314476,12.49073,12.16922,6.295895,4.246219,5.093925,2.580937,2.809403,2.679016
GSM847087,6.887037,5.581526,3.172550,6.125426,2.680152,5.531435,4.140237,4.099861,2.725907,2.742127,⋯,10.99153,10.464466,12.76326,12.37694,6.423731,4.282367,5.125491,2.547068,2.829246,2.599838


In [135]:
rownames(df) <- df$X

In [136]:
df <- df[,2:ncol(df)]

In [137]:
colnames(df) <- sub("^X", "", colnames(df))

In [139]:
library(hgu133plus2.db)
# Get gene symbol
x <- hgu133plus2SYMBOL
# Get the probe identifiers that are mapped to a gene name
mapped_probes <- mappedkeys(x)
# Convert to a list
xx <- as.list(x[mapped_probes])

In [140]:
# Transform matrix in a table
ref_gene <- do.call(rbind, xx)
ref_gene <- as.data.frame(ref_gene)
# Add column to get a reference between old and new gene
ref_gene$old_gene <- rownames(ref_gene)
colnames(ref_gene) <- c("new_gene", "old_gene")

In [143]:
head(ref_gene)

,old_gene,new_gene
,<chr>,<chr>
1,1007_s_at,DDR1
2,1053_at,RFC2
3,117_at,HSPA6
4,121_at,PAX8
5,1255_g_at,GUCA1A
6,1294_at,UBA7


In [142]:
# Make smaller table to speedup the translation
ref_gene_df <- data.frame(colnames(df))
colnames(ref_gene_df) <- c("old_gene")
ref_gene <- merge(x=ref_gene, y=ref_gene_df, by="old_gene")

In [144]:
# Get Gene Symbol from platform columns
new_col <- c()
for (i in colnames(df)) {
    new_col <- append(new_col, ref_gene[ref_gene$old_gene == i, 'new_gene'])
}

In [145]:
colnames(df) <- new_col

In [147]:
# Remove NA
df <- df[, colSums(is.na(df)) == 0]
# Remove columns with all 0s
df <- df[, colSums(df) > 0]
# Remove columns without gene symbol
df <- df[,nzchar(colnames(df))]
# Remove double names genes (GENE1 /// GENE2)
colnames(df) <- gsub(" ///.*", "", colnames(df))
# Make all columns uppercase
colnames(df) <- toupper(colnames(df))

In [148]:
# Remove columns which start by "NA" because they didn't refer to any gene
null_columns <- grep("NA.", colnames(df))
df <- df[,-null_columns]

In [149]:
head(df)

,DDR1,RFC2,HSPA6,PAX8,GUCA1A,UBA7,THRA,PTPN21,CCL5,CYP2E1,⋯,LRTM1.1,DGCR8.3,EXOSC4.2,EXOSC4.3,EHBP1L1.3,MEX3D.2,EPS8L1.4,BCAN.2,DCAF15.3,NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GSM847083,6.856013,4.872198,3.070721,5.949972,2.563389,6.549806,3.887666,4.335238,2.631582,2.742217,⋯,2.972538,4.109483,7.459844,2.687935,3.357634,4.267203,2.979519,3.592563,2.660471,3.964065
GSM847084,6.373692,5.160413,3.086563,6.187076,2.833957,6.397387,4.040880,4.335334,2.513761,2.724881,⋯,2.725198,4.607020,7.648045,2.909657,3.029686,4.463457,2.883185,3.483309,2.713319,3.801825
GSM847085,6.562429,5.359019,3.207338,6.280731,2.577091,5.999377,4.046283,4.446986,2.730223,2.905883,⋯,2.931048,4.219745,7.004625,2.834465,3.537678,4.186574,2.925019,3.745635,2.787650,3.865879
GSM847086,6.934789,6.031420,3.232655,6.416759,2.684375,5.246129,4.077676,4.198332,2.495319,2.808868,⋯,3.100757,4.260399,7.158476,2.514485,3.097804,3.906699,3.002416,3.451537,2.779323,3.787423
GSM847087,6.887037,5.581526,3.172550,6.125426,2.680152,5.531435,4.140237,4.099861,2.725907,2.742127,⋯,2.829036,4.432039,7.208206,2.717797,3.095362,4.369413,3.065409,3.533245,2.651866,3.919860


In [150]:
write.table(df, file=paste(dataPath, csv_name, sep=""), sep=",")

# GSD 

In [11]:
csv_name = 'GSD_GSE188956.csv'

In [12]:
df <- read.csv(file=paste(dataPath, csv_name, sep=""), header=TRUE)

In [13]:
head(df)

,hsa.let.7a.000377,hsa.let.7c.000379,hsa.let.7d.002283,hsa.let.7e.002406,hsa.let.7f.000382,hsa.let.7g.002282,hsa.miR.1.002222,hsa.miR.9.000583,hsa.miR.10a.000387,hsa.miR.10b.002218,⋯,hsa.miR.448.001029,hsa.miR.492.001039,hsa.miR.506.001050,hsa.miR.509.3.5p.002155,hsa.miR.511.001111,hsa.miR.517b.001152,hsa.miR.519c.001163,hsa.miR.520b.001116,hsa.miR.520e.001119,hsa.miR.520f.001120
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
G_107_GSP_144_no_tumor_19_M.txt,24.015,30.965,23.391,22.342,25.925,23.932,33.666,30.182,29.209,31.421,⋯,40,40,40,40,32.199,40,40.000,40.000,40,40.000
G_51_GSP_83_no_tumor_19_F.txt,24.655,28.127,24.655,22.682,28.316,25.332,31.168,31.613,27.041,25.775,⋯,40,40,40,40,32.259,40,40.000,40.000,40,40.000
G_34_GSP_54_no_tumor_19_M_BIS.txt,24.958,29.379,23.988,22.087,28.589,24.587,40.000,28.932,27.058,26.747,⋯,40,40,40,40,40.000,40,40.000,40.000,40,40.000
G_102_GSP_155_no_tumor_20_F.txt,23.746,27.057,22.650,21.119,26.293,23.270,33.221,29.778,26.117,26.834,⋯,40,40,40,40,32.986,40,40.000,34.255,40,31.311
G_32_GSP_51_no_tumor_21_F.txt,40.000,40.000,33.873,28.985,40.000,32.523,40.000,40.000,40.000,27.387,⋯,40,40,40,40,40.000,40,31.472,40.000,40,40.000
G_170_GSP_221_no_tumor_22_F.txt,24.899,27.147,22.932,22.080,27.598,22.797,31.694,29.518,26.336,27.628,⋯,40,40,40,40,29.842,40,40.000,35.581,40,40.000


In [7]:
library(miRBaseConverter)

In [15]:
checkMiRNAVersion(gsub("\\.", "-", colnames(df)))

   Version Proportion           Recommend
1       v6         0%                    
2     v7_1      0.26%  ***BEST Matched***
3       v8      0.26%  ***BEST Matched***
4     v8_1      0.26%  ***BEST Matched***
5     v8_2      0.26%  ***BEST Matched***
6       v9      0.26%  ***BEST Matched***
7     v9_1      0.26%  ***BEST Matched***
8     v9_2      0.26%  ***BEST Matched***
9      v10         0%                    
10   v10_1         0%                    
11     v11         0%                    
12     v12         0%                    
13     v13         0%                    
14     v14         0%                    
15     v15         0%                    
16     v16         0%                    
17     v17         0%                    
18     v18         0%                    
19     v19         0%                    
20     v20         0%                    
21     v21         0%                    
22     v22         0%                    


[1] "v9_2"

# Extra code

In [ ]:
library(org.Hs.eg.db)
columns(org.Hs.eg.db)
keys <- keys(org.Hs.eg.db)
ref_gene <- select(org.Hs.eg.db, keys=keys, columns = c("SYMBOL", "ENSEMBL"))

In [ ]:
library(hugene20stprobeset.db)
# Get gene symbol
x <- hugene20stprobesetSYMBOL
# Get the probe identifiers that are mapped to a gene name
mapped_probes <- mappedkeys(x)
# Convert to a list
xx <- as.list(x[mapped_probes])
# Transform matrix in a table
ref_gene <- do.call(rbind, xx)
ref_gene <- as.data.frame(ref_gene)
# Add column to get a reference between old and new gene
ref_gene$old_gene <- rownames(ref_gene)
colnames(ref_gene) <- c("new_gene", "old_gene")